In [1]:
from datasets import Dataset,DatasetDict
import os
import pandas as pd
import csv

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder_path = "C:/Users/Hector Auvinen/Desktop/UKP_sentential_argument_mining/data"
data = []

# names
"""col_names = ["topic", "retrievedUrl", "archivedUrl", "sentenceHash", "sentence", "annotation", "set"]
file_path = "C:/Users/Hector Auvinen/Desktop/UKP_sentential_argument_mining/data/school_uniforms.tsv"
df = pd.read_csv(file_path,sep="\t",header=None)
df"""

'col_names = ["topic", "retrievedUrl", "archivedUrl", "sentenceHash", "sentence", "annotation", "set"]\nfile_path = "C:/Users/Hector Auvinen/Desktop/UKP_sentential_argument_mining/data/school_uniforms.tsv"\ndf = pd.read_csv(file_path,sep="\t",header=None)\ndf'

In [3]:
for filename in os.listdir(folder_path):
    if filename.endswith(".tsv"):
        file_path = os.path.join(folder_path, filename)
        print(file_path.split("data")[-1])
        # Read the TSV file into a DataFrame
        df = pd.read_csv(file_path, sep='\t',quoting=csv.QUOTE_NONE)
        # Append data from the current file to the list
        print(len(df))
        data.append(df)

\abortion.tsv
3929
\cloning.tsv
3039
\death_penalty.tsv
3651
\gun_control.tsv
3341
\marijuana_legalization.tsv
2475
\minimum_wage.tsv
2473
\nuclear_energy.tsv
3576
\school_uniforms.tsv
3008


In [4]:
# Concatenate data from all files into one DataFrame
all_data = pd.concat(data, ignore_index=True)

# Convert the DataFrame into a Hugging Face dataset
dataset = Dataset.from_pandas(all_data)

In [5]:
# Split the dataset into train, validation, and test sets based on the 'set' column
train_dataset = dataset.filter(lambda example: example['set'] == 'train')
val_dataset = dataset.filter(lambda example: example['set'] == 'val')
test_dataset = dataset.filter(lambda example: example['set'] == 'test')

Filter:   0%|          | 0/25492 [00:00<?, ? examples/s]

Filter: 100%|██████████| 25492/25492 [00:00<00:00, 67263.51 examples/s]


In [6]:
# Remove the 'set' column from the datasets
train_dataset = train_dataset.remove_columns(["retrievedUrl","archivedUrl","sentenceHash"])
val_dataset = val_dataset.remove_columns(["retrievedUrl","archivedUrl","sentenceHash"])
test_dataset = test_dataset.remove_columns(["retrievedUrl","archivedUrl","sentenceHash"])

# Create a dictionary containing the train, validation, and test sets
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [7]:
# Check 'set' column values in train dataset
for example in dataset_dict["train"]:
    assert example["set"] == "train", "Incorrect value in 'set' column for train dataset"

# Check 'set' column values in validation dataset
for example in dataset_dict["validation"]:
    assert example["set"] == "val", "Incorrect value in 'set' column for validation dataset"

# Check 'set' column values in test dataset
for example in dataset_dict["test"]:
    assert example["set"] == "test", "Incorrect value in 'set' column for test dataset"

In [9]:
# Display the first few examples from each split
print("Train dataset:")
print(dataset_dict["train"][0])
print("\nValidation dataset:")
print(dataset_dict["validation"][0])
print("\nTest dataset:")
print(dataset_dict["test"][0])

Train dataset:
{'topic': 'abortion', 'sentence': 'Where did you get that ?', 'annotation': 'NoArgument', 'set': 'train'}

Validation dataset:
{'topic': 'abortion', 'sentence': 'This means it has to steer monetary policy to ( a ) keep prices stable , and to ( b ) keep unemployment low and the economy growing .', 'annotation': 'NoArgument', 'set': 'val'}

Test dataset:
{'topic': 'abortion', 'sentence': 'With that I would like to give everyone something to contemplate .', 'annotation': 'NoArgument', 'set': 'test'}


In [10]:
write_path = "C:/Users/Hector Auvinen/Desktop/UKP_sentential_argument_mining/hf_data/argument_mining"

dataset_dict.save_to_disk(write_path)

Saving the dataset (1/1 shards): 100%|██████████| 5109/5109 [00:00<00:00, 75333.53 examples/s]


In [11]:
from datasets import load_dataset,load_from_disk
loaded_dataset = load_from_disk(write_path)

In [12]:
loaded_dataset

DatasetDict({
    train: Dataset({
        features: ['topic', 'sentence', 'annotation', 'set'],
        num_rows: 18341
    })
    validation: Dataset({
        features: ['topic', 'sentence', 'annotation', 'set'],
        num_rows: 2042
    })
    test: Dataset({
        features: ['topic', 'sentence', 'annotation', 'set'],
        num_rows: 5109
    })
})